<a href="https://colab.research.google.com/github/Noran69/Exercises-Assignments/blob/main/4__exercise_grouping_and_sorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In these exercises we'll apply groupwise analysis to our dataset.

Run the code cell below to load the data before running the exercises.

In [2]:
import pandas as pd

reviews = pd.read_csv("/content/sample_data/winemag-data_first150k.csv", index_col=0)
#pd.set_option("display.max_rows", 5)
reviews.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


# Exercises

## 1.
Who are the most common wine reviewers in the dataset? Create a `Series` whose index is the `taster_twitter_handle` category from the dataset, and whose values count how many reviews each person wrote.

In [11]:
# Your code here
reviews_written = reviews.groupby('winery').size()
reviews_written

,0
winery,
'37 Cellars,2
1+1=3,10
10 Knots,16
1000 Stories,1
1040FU,1
...,...
Ñandú,3
Único Luis Miguel,2
àMaurice,44


## 2.
What is the best wine I can buy for a given amount of money? Create a `Series` whose index is wine prices and whose values is the maximum number of points a wine costing that much was given in a review. Sort the values by price, ascending (so that `4.0` dollars is at the top and `3300.0` dollars is at the bottom).

In [12]:
best_rating_per_price = reviews.groupby('price')['points'].max().sort_index()
best_rating_per_price


,points
price,
4.0,86
5.0,90
6.0,90
7.0,90
8.0,91
...,...
1300.0,96
1400.0,100
1900.0,98


## 3.
What are the minimum and maximum prices for each `variety` of wine? Create a `DataFrame` whose index is the `variety` category from the dataset and whose values are the `min` and `max` values thereof.

In [13]:
price_extremes = reviews.groupby('variety').price.agg([min, max])
price_extremes



<ipython-input-13-3b16e88edd3a>:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  price_extremes = reviews.groupby('variety').price.agg([min, max])
<ipython-input-13-3b16e88edd3a>:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  price_extremes = reviews.groupby('variety').price.agg([min, max])


,min,max
variety,,
Agiorgitiko,8.0,65.0
Aglianico,6.0,130.0
Aidani,27.0,27.0
Airen,8.0,10.0
Albana,8.0,66.0
...,...,...
Zierfandler-Rotgipfler,20.0,25.0
Zinfandel,4.0,100.0
Zlahtina,13.0,17.0


## 4.
What are the most expensive wine varieties? Create a variable `sorted_varieties` containing a copy of the dataframe from the previous question where varieties are sorted in descending order based on minimum price, then on maximum price (to break ties).

In [14]:
sorted_varieties = price_extremes.sort_values(by=['min', 'max'], ascending=False)
sorted_varieties


,min,max
variety,,
Cabernet-Shiraz,150.0,150.0
Mazuelo,92.0,105.0
Carignan-Syrah,80.0,80.0
Syrah-Cabernet Franc,69.0,69.0
Nasco,65.0,65.0
...,...,...
Rabigato,NaN,NaN
Sacy,NaN,NaN
Sauvignon Blanc-Sauvignon Gris,NaN,NaN


## 5.
Create a `Series` whose index is reviewers and whose values is the average review score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns.

In [16]:
reviewer_mean_ratings = reviews.groupby('winery').points.mean()
reviewer_mean_ratings



,points
winery,
'37 Cellars,88.000000
1+1=3,83.800000
10 Knots,84.375000
1000 Stories,90.000000
1040FU,82.000000
...,...
Ñandú,83.333333
Único Luis Miguel,85.000000
àMaurice,91.477273


Are there significant differences in the average scores assigned by the various reviewers? Run the cell below to use the `describe()` method to see a summary of the range of values.

In [17]:
reviewer_mean_ratings.describe()

,points
count,14810.000000
mean,87.370020
std,2.685948
min,80.000000
25%,85.500000
50%,87.285714
75%,89.114316
max,100.000000


## 6.
What combination of countries and varieties are most common? Create a `Series` whose index is a `MultiIndex`of `{country, variety}` pairs. For example, a pinot noir produced in the US should map to `{"US", "Pinot Noir"}`. Sort the values in the `Series` in descending order based on wine count.

In [18]:
country_variety_counts = reviews.groupby(['country', 'variety']).size().sort_values(ascending=False)
country_variety_counts



country      variety                 
US           Pinot Noir                  10340
             Cabernet Sauvignon           9178
             Chardonnay                   8127
France       Bordeaux-style Red Blend     4960
US           Syrah                        4274
                                         ...  
France       Pied de Perdrix                 1
Tunisia      White Blend                     1
             Rosé                            1
Switzerland  White Blend                     1
US           Carignan-Grenache               1
Length: 1475, dtype: int64

# Keep going
